In [2]:
#import the libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [3]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [4]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df.dropna(axis=0, inplace=True)

In [9]:
df.shape

(18285, 5)

In [10]:
#download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Albert\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
#Create a function to process the text
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

    return clean_words

In [14]:
#apply this function
df['text'].head().apply(process_text)

0    [House, Dem, Aide, Didn’t, Even, See, Comey’s,...
1    [Ever, get, feeling, life, circles, roundabout...
2    [Truth, Might, Get, Fired, October, 29, 2016, ...
3    [Videos, 15, Civilians, Killed, Single, US, Ai...
4    [Print, Iranian, woman, sentenced, six, years,...
Name: text, dtype: object

In [15]:
df['text']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 18285, dtype: object

The next step (creating the bag of words) took me around an hour

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer = process_text).fit_transform(df['text'])

In [18]:
#split the data into 80% training datasets and 20% testing datasets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow, df['label'], test_size = 0.20, random_state = 0)

In [19]:
#we want to get the shape of message bow(bag of words). We want to get the  number of rows and the mumber of columns.
message_bow.shape

(18285, 280080)

In [20]:
#create and train a naive bayes classifier model
from sklearn.naive_bayes import MultinomialNB
classifier_NB = MultinomialNB()
classifier_NB.fit(X_train, y_train)

MultinomialNB()

In [21]:
#now we will print the predictions. 
print(classifier_NB.predict(X_train))
print(y_train.values)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]


In [22]:
#Now we can't see all the data, but we can see the first three and last rows. 
#The model classified that the first row of data with the value 1 (which were sorted as fake news) are indeed fake news. Except the third, that the model thougth it was real and it resulted in fake. 
#The last numbers show a correspondency between what the model thinks it is real or fake and what they actually are. 

In [23]:
#Let's evaluate the model on the training data set. we are going to use Naive Bayes Classifier for this.
from sklearn.metrics import classification_report
pred_NB = classifier_NB.predict(X_train)
print(classification_report(y_train, pred_NB))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      8321
           1       0.99      0.94      0.97      6307

    accuracy                           0.97     14628
   macro avg       0.98      0.97      0.97     14628
weighted avg       0.97      0.97      0.97     14628



In [24]:
#now we will see how the dataset do on the test dataset wich should be data that it has never seen before.

from sklearn.metrics import classification_report
pred_NB = classifier_NB.predict(X_test)
print(classification_report(y_test, pred_NB))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      2040
           1       0.99      0.90      0.94      1617

    accuracy                           0.95      3657
   macro avg       0.96      0.95      0.95      3657
weighted avg       0.95      0.95      0.95      3657



In [ ]:
#we see that the accuracy has went down a little to 95%

In [25]:
NB_score = classifier_NB.score(X_test, y_test)
print(NB_score)

0.9521465682253213


In [26]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier_LR = LogisticRegression(C=1e5)
classifier_LR.fit(X_train, y_train)

C:\Users\Albert\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0)

In [27]:
#now we will print the predictions. 
print(classifier_LR.predict(X_train))
print(y_train.values)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]


In [28]:
pred_LR = classifier_LR.predict(X_train)
print(classification_report(y_train, pred_LR))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8321
           1       1.00      1.00      1.00      6307

    accuracy                           1.00     14628
   macro avg       1.00      1.00      1.00     14628
weighted avg       1.00      1.00      1.00     14628



In [29]:
pred_LR = classifier_LR.predict(X_test)
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2040
           1       0.98      0.98      0.98      1617

    accuracy                           0.99      3657
   macro avg       0.98      0.99      0.99      3657
weighted avg       0.99      0.99      0.99      3657



In [30]:
LR_score = classifier_LR.score(X_test, y_test)
print(LR_score)

0.985233798195242


In [31]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier_RF = RandomForestClassifier(n_estimators=5)
classifier_RF.fit(X_train, y_train)

RandomForestClassifier(n_estimators=5)

In [32]:
#now we will print the predictions. 
print(classifier_LR.predict(X_train))
print(y_train.values)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]


In [33]:
pred_RF = classifier_RF.predict(X_train)
print(classification_report(y_train, pred_RF))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      8321
           1       0.99      0.98      0.99      6307

    accuracy                           0.99     14628
   macro avg       0.99      0.99      0.99     14628
weighted avg       0.99      0.99      0.99     14628



In [34]:
pred_RF = classifier_RF.predict(X_test)
print(classification_report(y_test, pred_RF))

              precision    recall  f1-score   support

           0       0.85      0.92      0.89      2040
           1       0.89      0.80      0.84      1617

    accuracy                           0.87      3657
   macro avg       0.87      0.86      0.86      3657
weighted avg       0.87      0.87      0.87      3657



In [35]:
RF_score = classifier_RF.score(X_test, y_test)
print(RF_score)

0.8676510801203172


In [36]:
#Resultados finales
print("La precisión del clasificador Random Forest es del {}%".format(RF_score*100))
print("La precisión del clasificador Naive Bayes es del {}%".format(NB_score*100))
print("La precisión del clasificador Logistic Regression es del {}%".format(LR_score*100))

La precisión del clasificador Random Forest es del 86.76510801203172%
La precisión del clasificador Naive Bayes es del 95.21465682253213%
La precisión del clasificador Logistic Regression es del 98.52337981952421%


As We can see, our best classifier is Logistic Regression.